# Evaluating Synthetic Data Generators

A very common question when someone starts using **SDV** to generate
synthetic data is: _"How good is the data that I just generated?"_

In order to answer this question, **SDV** has a collection of metrics and tools
that allow you to compare the _real_ that you provided and the _synthetic_ data
that you generated using **SDV** or any other tool and compute a series of
scores that indicate how similar they are.

In this guide we will show you how to perform this evaluation and how to explore
the different metrics that exist.

## Using the SDV Evaluation Framework

In order to be able to evaluate the quality of synthetic data we essentially need
two things: _real_ data and _synthetic_ data that pretends to resemble it.

Let us start by loading a demo table and generate a synthetic replica of it
using the `GaussianCopula` model.

In [1]:
# Setup logging and warnings - change ERROR to INFO for increased verbosity
import logging
logging.basicConfig(level=logging.ERROR)

logging.getLogger().setLevel(level=logging.ERROR)
logging.getLogger('sdv').setLevel(level=logging.ERROR)

import warnings
warnings.simplefilter("ignore")

In [2]:
from sdv.demo import load_tabular_demo
from sdv.tabular import GaussianCopula

real_data = load_tabular_demo('student_placements')

model = GaussianCopula()
model.fit(real_data)
synthetic_data = model.sample()

After the previous steps we will have two tables:

- `real_data`: A table containing data about student placements

In [3]:
real_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


- `synthetic_data`: A synthetically generated table that contains data in the
  same format and with similar statistical properties as the `real_data`.  

In [4]:
synthetic_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17289,M,71.043275,65.596746,Science,66.154880,Sci&Tech,False,0,59.676346,Mkt&Fin,72.167106,NaN,False,NaT,NaT,3.0
1,17456,F,68.972675,67.636077,Science,72.105033,Comm&Mgmt,False,0,93.091771,Mkt&Fin,72.902805,30574.131054,True,2020-04-22 03:21:03.285731328,2020-10-16 19:37:20.150722048,3.0
2,17275,F,82.401161,66.447492,Science,67.663266,Sci&Tech,False,1,64.064089,Mkt&HR,62.209602,31171.007086,True,2020-05-14 09:00:55.651516160,2020-08-11 06:14:06.541445632,3.0
3,17451,M,82.445156,72.090023,Science,68.648009,Comm&Mgmt,False,1,83.277989,Mkt&HR,64.435476,27798.642877,True,2020-03-22 12:45:32.649371392,2020-09-20 05:00:42.626356736,3.0
4,17293,F,73.962380,71.040811,Science,80.141950,Sci&Tech,False,0,53.361144,Mkt&HR,64.375579,25492.739664,True,2020-04-09 06:14:39.522265088,2020-11-25 15:00:49.246789120,3.0


.. note:: For more details about this process, please visit the :ref:`gaussian_copula` guide.

### Computing an overall score

The simplest way to see how similar the two tables are is to import the `sdv.evaluation.evaluate`
function and run it passing both the `synthetic_data` and the `real_data` tables.

In [5]:
from sdv.evaluation import evaluate

evaluate(synthetic_data, real_data)

0.7301232811530844

The output of this function call will be a number between 0 and 1 that will
indicate us how similar the two tables are, being 0 the worst and 1 the best
possible score.

### How was the obtained score computed?

The `evaluate` function applies a collection of pre-configured metric functions and returns
the average of the scores that the data obtained on each one of them. In most scenarios this
can be enough to get an idea about the similarity of the two tables, but you might want to
explore the metrics in more detail.

In order to see the different metrics that were applied you can pass and additional argument
`aggregate=False`, which will make the `evaluate` function return a dictionary with the scores
that each one of the metrics functions returned:

In [6]:
evaluate(synthetic_data, real_data, aggregate=False)

{'cstest': 0.8738863767138708,
 'kstest': 0.4899793678776011,
 'logistic_detection': 0.764000282559555,
 'svc_detection': 0.7709655567147742}

### Can I control which metrics are applied?

By default, the `evaluate` function will apply all the metrics that are included within
the SDV Evaluation framework. However, the list of metrics that are applied can be controlled
by passing a list with the names of the metrics that you want to apply.

For example, if you were interested on obtaining only the `cstest` and `kstest` metrics
you can call the `evaluate` function as follows:

In [7]:
evaluate(synthetic_data, real_data, metrics=['cstest', 'kstest'])

0.681932872295736

Or, if we want to see the scores separately:

In [8]:
evaluate(synthetic_data, real_data, metrics=['cstest', 'kstest'], aggregate=False)

{'cstest': 0.8738863767138708, 'kstest': 0.4899793678776011}

The complete list of possible metrics is:

* `cstest`: This metric compares the distributions of all the categorical
  columns of the table by using a Chi-squared test and returns the average of
  the `p-values` obtained across all the columns. If the tables that you are
  evaluating do not contain any categorical columns the result will be `nan`.
* `kstest`: This metric compares the distributions of all the numerical columns
  of the table with a two-sample Kolmogorov–Smirnov test using the empirical CDF
  and returns the average of the `p-values` obtained across all the columns.
  If the tables that you are evaluating do not contain any numerical columns the
  result will be `nan`.
* `logistic_detection`: This metric tries to use a Logistic Regression classifier
  to detect whether each row is real or synthetic and then evaluates its performance
  using an Area under the ROC curve metric.
* `svc_detection`: This metric tries to use an Support Vector Classifier
  to detect whether each row is real or synthetic and then evaluates its performance
  using an Area under the ROC curve metric.